<a href="https://colab.research.google.com/github/emamsayed/emamsaye/blob/main/sentimental_analysis_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"emamsayed","key":"b9ac91a1e76465f2c83d2b9be7cedc31"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                       title                                   size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                           DAIGT V2 Train Dataset                  29MB  2023-11-16 01:38:36           1284        140  1.0              
muhammadbinimran/housing-price-prediction-data            Housing Price Prediction Data          763KB  2023-11-21 17:56:32           5181         95  1.0              
thedevastator/netflix-imdb-scores                         Netflix IMDB Scores                    699KB  2023-12-03 14:10:34           1244         24  1.0              
youssefaboelwafa/hotel-booking-cancellation-prediction    Hotel Booking Cancellation Prediction  470KB  2023-12-03 01:54:35            805         24  1.0 

In [ ]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 131MB/s]


In [ ]:
! unzip twitter-entity-sentiment-analysis.zip

Archive:  twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
pd.options.mode.chained_assignment = None
import os #File location
from wordcloud import WordCloud #Word visualization
import matplotlib.pyplot as plt #Plotting properties
import seaborn as sns #Plotting properties
from sklearn.feature_extraction.text import CountVectorizer #Data transformation
from sklearn.model_selection import train_test_split #Data testing
from sklearn.ensemble import RandomForestClassifier #Prediction Model
from sklearn.metrics import accuracy_score, f1_score #Comparison between real and predicted
import re #Regular expressions
import nltk
from nltk import word_tokenize
import time
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Validation dataset
val=pd.read_csv("twitter_validation.csv", header=None)
#Full dataset for Train-Test
train=pd.read_csv("twitter_training.csv", header=None)

In [ ]:
train.columns=['id','information','type','text']
train.head()

,id,information,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
val.columns=['id','information','type','text']
val.head()

,id,information,type,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [ ]:
# Assuming df is your DataFrame and 'label' is the column containing the labels
train = train[train['type'] != 'Irrelevant']
val = val[val['type'] != 'Irrelevant']

In [ ]:
train_data=train #[(train["type"] == "Positive") | (train["type"] == "Negative")]
train_data

,id,information,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [ ]:
val_data=val #[(val["type"] == "Positive") | (val["type"] == "Negative")]
val_data

,id,information,type,text
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
5,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
993,314,Amazon,Negative,Please explain how this is possible! How can t...
994,9701,PlayStation5(PS5),Positive,Good on Sony. As much as I want to see the new...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [ ]:
#Text transformation
train_data["lower"]=train_data.text.str.lower() #lowercase
train_data["lower"]=[str(data) for data in train_data.lower] #converting all to string
train_data["lower"]=train_data.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex
val_data["lower"]=val_data.text.str.lower() #lowercase
val_data["lower"]=[str(data) for data in val_data.lower] #converting all to string
val_data["lower"]=val_data.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex

In [ ]:
train_data.head()

,id,information,type,text,lower
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...


In [ ]:
#Count information per category
plot1=train.groupby(by=["information","type"]).count().reset_index()
plot1.head()

,information,type,id,text,lower
0,Amazon,Negative,576,575,576
1,Amazon,Neutral,1236,1207,1236
2,Amazon,Positive,312,308,312
3,ApexLegends,Negative,600,591,600
4,ApexLegends,Neutral,942,936,942


In [ ]:
nltk.download('punkt')

#Text splitting
tokens_text = [word_tokenize(str(word)) for word in train_data.lower]
#Unique word counter
tokens_counter = [item for sublist in tokens_text for item in sublist]
print("Number of tokens: ", len(set(tokens_counter)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of tokens:  26396


In [ ]:
tokens_text[1]

['i',
 'am',
 'coming',
 'to',
 'the',
 'borders',
 'and',
 'i',
 'will',
 'kill',
 'you',
 'all']

In [ ]:
#Choosing english stopwords
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')
stop_words[:5]

['i', 'me', 'my', 'myself', 'we']

In [ ]:
#Initial Bag of Words
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    stop_words=stop_words, #English Stopwords
    ngram_range=(1, 1) #analysis of one word
)

In [ ]:
#Train - Test splitting
reviews_train, reviews_test = train_test_split(train_data, test_size=0.2, random_state=0)

In [ ]:
#Creation of encoding related to train dataset
X_train_bow = bow_counts.fit_transform(reviews_train.lower)
#Transformation of test dataset with train encoding
X_test_bow = bow_counts.transform(reviews_test.lower)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


In [ ]:
#Labels for train and test encoding
y_train_bow = reviews_train['type']
y_test_bow = reviews_test['type']

In [ ]:
#Total of registers per category
y_test_bow.value_counts() / y_test_bow.shape[0]

Negative    0.372478
Positive    0.334306
Neutral     0.293217
Name: type, dtype: float64

In [ ]:
# Random Forest
start=time.time()
model1 = RandomForestClassifier()
model1.fit(X_train_bow, y_train_bow)
end=time.time()
# Prediction
test_pred = model1.predict(X_test_bow)
tim=end-start
print("Training time: ",tim)
print("Accuracy: ", accuracy_score(y_test_bow, test_pred) * 100)

Training time:  99.33133625984192
Accuracy:  92.00907691060864


In [ ]:
# F1 score
f1 = f1_score(y_test_bow, test_pred, average='weighted')
print("F1 Score: ", f1)

F1 Score:  0.9201464020359419


In [ ]:
# Assuming 'new_review' contains the new review you want to predict
new_review = "I hate it"

# 2. Transform the new review using the existing 'bow_counts' and make a prediction
new_review_bow = bow_counts.transform([new_review.lower()])  # Transform the new review
predicted_sentiment = model1.predict(new_review_bow)  # Make a prediction

print("Predicted Sentiment:", predicted_sentiment)

Predicted Sentiment: ['Negative']


In [ ]:
#Validation data
X_val_bow = bow_counts.transform(val_data.lower)
y_val_bow = val_data['type']

In [ ]:
Val_res = model1.predict(X_val_bow)
print("Accuracy: ", accuracy_score(y_val_bow, Val_res) * 100)

Accuracy:  96.01449275362319


In [ ]:
#n-gram of 4 words
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    ngram_range=(1,4)
)
#Data labeling
X_train_bow = bow_counts.fit_transform(reviews_train.lower)
X_test_bow = bow_counts.transform(reviews_test.lower)
X_val_bow = bow_counts.transform(val_data.lower)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
model2 = LogisticRegression(C=0.9, solver="liblinear", multi_class="ovr")    #one vs rest(All) method for multi classification
# Logistic regression
start=time.time()
model2.fit(X_train_bow, y_train_bow)
end=time.time()
tim=end-start
# Prediction
test_pred_2 = model2.predict(X_test_bow)
print("Training time: ", tim)
print("Accuracy: ", accuracy_score(y_test_bow, test_pred_2) * 100)

NameError: ignored

In [ ]:
y_val_bow = val_data['type']
Val_pred_2 = model2.predict(X_val_bow)
print("Accuracy: ", accuracy_score(y_val_bow, Val_pred_2) * 100)

Accuracy:  98.79227053140096


In [ ]:
# F1 score
f1 = f1_score(y_test_bow, test_pred_2, average='weighted')
print("F1 Score: ", f1)

F1 Score:  0.9186938449212562


In [ ]:
# Assuming 'new_review' contains the new review you want to predict
new_review = " so far so good "

# 2. Transform the new review using the existing 'bow_counts' and make a prediction
new_review_bow = bow_counts.transform([new_review.lower()])  # Transform the new review
predicted_sentiment = model2.predict(new_review_bow)  # Make a prediction

print("Predicted Sentiment:", predicted_sentiment)

Predicted Sentiment: ['Positive']


In [ ]:
import pickle

pickle.dump(model2, open('/content/model_saved2', 'wb'))

pickle.dump(bow_counts, open('/content/model_vector3', 'wb'))

In [ ]:
test=pd.read_excel("Amazon.xlsx")

In [ ]:
test.columns=['product','review','type','link']

# Convert the 'review' column to lowercase
test['review'] = test['review'].str.lower()

# Convert all values in the 'review' column to strings
test['review'] = test['review'].astype(str)

# Remove non-alphanumeric characters using a regular expression
test['review'] = test['review'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

NameError: ignored

In [ ]:
test['review']

0                                  good dual dash camera
1                                         so far so good
2                                    drains your battery
3      tucson az  freezes on hot days   multiple issu...
4                    don t bother with this buggy camera
                             ...                        
414    on my third pair of ghost 14 s provided i stre...
415    loved these at first but after 9 or ten wearin...
416    not really sure what happened with these shoes...
417                  struggling to find marathon shoes  
418    a few years ago  i went to a specialist store ...
Name: review, Length: 419, dtype: object

In [ ]:
# Convert the 'review' column to lowercase
test['type'] = test['type'].str.lower()

# Convert all values in the 'review' column to strings
test['type'] = test['type'].astype(str)

# Remove non-alphanumeric characters using a regular expression
test['type'] = test['type'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

In [ ]:
y_true=test['type']
y_true

0      positive
1      positive
2      negative
3      negative
4      negative
         ...   
414    positive
415    negative
416    negative
417     neutral
418    positive
Name: type, Length: 419, dtype: object

In [ ]:
predictions = []
y_true = test['type']
count = 0
i=0
# Iterate through the 'review' column in the 'test' DataFrame
for review in test['review']:
    # Transform the individual review using the BoW vectorizer
    review_bow = bow_counts.transform([review])

    # Make predictions using model2 for the individual review
    review_prediction = model2.predict(review_bow)
    review_prediction_str = review_prediction[0]  # Get the prediction as a string

    if review_prediction_str.lower() == 'positive':
        review_prediction_str = "positive"
    elif review_prediction_str.lower() == 'negative':
        review_prediction_str = 'negative'
    else:
        review_prediction_str = "neutral"

    # Append the prediction to the list of predictions
    predictions.append(review_prediction_str)

    if review_prediction_str == y_true[i]:
        count += 1
    i+=1

# Print the count and accuracy
print("Correct Predictions:", count)
print("Accuracy:", count / len(predictions))

# Now, 'predictions' contains the predictions for each review in the 'test' DataFrame
print(predictions)

Correct Predictions: 224
Accuracy: 0.5346062052505967
['positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'neutral', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'positiv

In [ ]:
# Filter the DataFrame to keep rows where "prediction" is not equal to "neutral"
df = test[test['type'] != 'neutral']
df = df[test['type'] != 'natural']
df

<ipython-input-53-6d598285314f>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[test['type'] != 'natural']


,product,review,type,link
0,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",good dual dash camera,positive,NaN
1,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",so far so good,positive,NaN
2,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",drains your battery,negative,NaN
3,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",tucson az freezes on hot days multiple issu...,negative,NaN
4,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",don t bother with this buggy camera,negative,NaN
...,...,...,...,...
413,Brooks Men's Ghost 14 Neutral Running Shoe,overall the shoe is quite comfortable to walk ...,negative,NaN
414,Brooks Men's Ghost 14 Neutral Running Shoe,on my third pair of ghost 14 s provided i stre...,positive,NaN
415,Brooks Men's Ghost 14 Neutral Running Shoe,loved these at first but after 9 or ten wearin...,negative,NaN
416,Brooks Men's Ghost 14 Neutral Running Shoe,not really sure what happened with these shoes...,negative,NaN


In [ ]:
predictions = []
y_true = df['type']
count = 0
i = 0

# Iterate through the 'review' column in the DataFrame
for review in df['review']:
    # Transform the individual review using the BoW vectorizer
    review_bow = bow_counts.transform([review])

    # Make predictions using model2 for the individual review
    review_prediction = model2.predict(review_bow)
    review_prediction_str = review_prediction[0].lower()  # Convert to lowercase

    # Convert numeric predictions to strings
    if review_prediction_str == 'positive':
        review_prediction_str = 'positive'
    elif review_prediction_str == 'negative':
        review_prediction_str = 'negative'

    # Append the prediction to the list of predictions
    predictions.append(review_prediction_str)

    # Compare the prediction to the true label
    if review_prediction_str == str(y_true.iloc[i]):
        count += 1
    i += 1

# Print the count and accuracy
print("Correct Predictions:", count)
print("Accuracy:", count / len(predictions))

# Now, 'predictions' contains the predictions for each review in the DataFrame
print(predictions)

Correct Predictions: 207
Accuracy: 0.6253776435045317
['positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'neutral', 'positive', 'positive', 'negative', 'negative', 'negative', 'neutral', 'positive', 'positive', 'positive', 'negative', 'neutral', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'ne